In [22]:
# Create Lat and Long Combos
# Import the dependencies.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import requests
from datetime import datetime

In [23]:
# Create a set of random latitude and longitude combinations.
lats = np.random.uniform(low=-90.000, high=90.000, size=1500)
lngs = np.random.uniform(low=-180.000, high=180.000, size=1500)
lat_lngs = zip(lats, lngs)
lat_lngs

#This zip objects packs each pair of lats and lngs having the same index in their respective array into a tuple


In [24]:
# Add the latitudes and longitudes to a list.
coordinates = list(lat_lngs)

In [25]:
# Use the citipy module to determine city based on latitude and longitude.
from citipy import citipy

!pip install gmaps

In [26]:
# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)

649

# 2.6 Retreive City Weather Data 

In [27]:
# Import the requests library.
import requests
from citipy import citipy

# Import the API key.
from config import weather_api_key
# Starting URL for Weather Map API Call.
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key
print(url)


ImportError: cannot import name 'weather_api_key' from 'config' (C:\Users\ddickson\DataBootCamp\World_Weather_Analysis\config.py)

In [ ]:
# Create an empty list to hold the weather data.
city_data = []
# Print the beginning of the logging.
#print("Beginning Data Retrieval     ")
#print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1

# Loop through all the cities in the list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1

    #Code Breakdown
    #1. create a for loop with enumerate() method and reference the index and the city in the list
    #2. Check if remainder of index divide by 50 equals 0 and if index is greater than or equal to 50. If statement true
    # then set_count and record_count are incremented by 1
    #3. inside coniditonal statement we create URL endpoint for each city as before, we are removing blank spaces in city name
    # and concatenating city name with city.replace(" ","+") which will find corresponding weather data for city instead of data
    #for first part of city name
    #4. add print statement that tells us record count, set count and city being processed
    #5. add one to the record count before next city is processed
 

# Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Date": city_date})

    # If an error is experienced, skip the city.
    #except:
       # print("City not found. Skipping...")
       # pass

# Indicate that Data Loading is complete.
#print("-----------------------------")
#print("Data Retrieval Complete      ")
#print("-----------------------------")

In [ ]:
#2.7 Create a Data Frame
# Convert the array of dictionaries to a Pandas DataFrame.
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

In [ ]:
# Set new column order
new_column_order = ["City", "Country", "Date", "Lat", "Lng", "Max Temp", "Humidity", "Cloudiness", "Wind Speed"]
# Set dataframe to accept new column order
city_data_df = city_data_df[new_column_order]
city_data_df.head(10)

In [ ]:
# Create the output file (CSV).
output_data_file = "weather_data/cities.csv"
# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")

# 3 Plot Lat vs Temp

In [ ]:
# Extract relevant fields from the DataFrame for plotting.
lats = city_data_df["Lat"]
max_temps = city_data_df["Max Temp"]
humidity = city_data_df["Humidity"]
cloudiness = city_data_df["Cloudiness"]
wind_speed = city_data_df["Wind Speed"]


In [ ]:
# Import time module
import time

# Build the scatter plot for latitude vs. max temperature.
plt.scatter(lats,
            max_temps,
            edgecolor="black", linewidths=1, marker="o",
            alpha=0.8, label="Cities")

# Incorporate the other graph properties.
plt.title(f"City Latitude vs. Max Temperature "+ time.strftime("%x"))
plt.ylabel("Max Temperature (F)")
plt.xlabel("Latitude")
plt.grid(True)

# Save the figure.
plt.savefig("weather_data/Fig1.png")

# Show plot.
plt.show()

In [ ]:
#3.2 Plot Lat vs Humidity
# Build the scatter plots for latitude vs. humidity.
plt.scatter(lats,
            humidity,
            edgecolor="black", linewidths=1, marker="o",
            alpha=0.8, label="Cities")

# Incorporate the other graph properties.
plt.title(f"City Latitude vs. Humidity "+ time.strftime("%x"))
plt.ylabel("Humidity (%)")
plt.xlabel("Latitude")
plt.grid(True)
# Save the figure.
plt.savefig("weather_data/Fig2.png")
# Show plot.
plt.show()

In [ ]:
#3.3 Plot Latitude vs Cloudiness
# Build the scatter plots for latitude vs. cloudiness.
plt.scatter(lats,
            cloudiness,
            edgecolor="black", linewidths=1, marker="o",
            alpha=0.8, label="Cities")

# Incorporate the other graph properties.
plt.title(f"City Latitude vs. Cloudiness (%) "+ time.strftime("%x"))
plt.ylabel("Cloudiness (%)")
plt.xlabel("Latitude")
plt.grid(True)
# Save the figure.
plt.savefig("weather_data/Fig3.png")
# Show plot.
plt.show()

In [ ]:
#3.4 Plot Lat vs Wind Speed
# Build the scatter plots for latitude vs. wind speed.
plt.scatter(lats,
            wind_speed,
            edgecolor="black", linewidths=1, marker="o",
            alpha=0.8, label="Cities")

# Incorporate the other graph properties.
plt.title(f"City Latitude vs. Wind Speed "+ time.strftime("%x"))
plt.ylabel("Wind Speed (mph)")
plt.xlabel("Latitude")
plt.grid(True)
# Save the figure.
plt.savefig("weather_data/Fig4.png")
# Show plot.
plt.show()

# 4 USE LINEAR REGRESSION TO FIND THE RELATIONSHIP BTW VARIABLES

In [ ]:
# Refer to random_numbers and linear regression for practice
#4.2 Find correlation btw lattitude and Max Temp and generate a regression line for North and South Hemispheres
# Create a function to create perform linear regression on the weather data
# and plot a regression line and the equation with the data.
#def plot_linear_regression(x_values, y_values, title, y_label, text_coordinates):

In [ ]:
# Import linregress
from scipy.stats import linregress

# Create a function to create perform linear regression on the weather data
# and plot a regression line and the equation with the data.
def plot_linear_regression(x_values, y_values, title, y_label, text_coordinates):

    # Run regression on hemisphere weather data.
    (slope, intercept, r_value, p_value, std_err) = linregress(x_values, y_values)

    # Calculate the regression line "y values" from the slope and intercept.
    regress_values = x_values * slope + intercept
    # Get the equation of the line.
    line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
    # Create a scatter plot and plot the regression line.
    plt.scatter(x_values,y_values)
    plt.plot(x_values,regress_values,"r")
    # Annotate the text for the line equation.
    plt.annotate(line_eq, text_coordinates, fontsize=15, color="red")
    plt.xlabel('Latitude')
    plt.ylabel(y_label)
    plt.show()

In [ ]:
# there is no output, we need to call the function with five parameters


In [ ]:
#Create the Hemisphere Data Frames to perform regresssion analysis on the max temps
# 1. Create a Norther Hemisphere Data Frame from city_data_df using loc
#row = df.loc[row_index]
index13 = city_data_df.loc[13]
index13

In [ ]:
# get all northern hemisphere lattitudes for lats >=0
city_data_df["Lat"] >= 0

In [ ]:
# get DataFrame with all data fitting critereia for lats >= 0 use loc()
city_data_df.loc[(city_data_df["Lat"] >= 0)].head()


In [ ]:
# assign above DF to new DF
# Create Northern and Southern Hemisphere DataFrames.
northern_hemi_df = city_data_df.loc[(city_data_df["Lat"] >= 0)]
southern_hemi_df = city_data_df.loc[(city_data_df["Lat"] < 0)]


In [ ]:
#Perform Linear Regression on the Maximum Temperature for the Northern Hemisphere
# Linear regression on the Northern Hemisphere
x_values = northern_hemi_df["Lat"]
y_values = northern_hemi_df["Max Temp"]
# Call the function.
plot_linear_regression(x_values, y_values,
                       'Linear Regression on the Northern Hemisphere \
                        for Maximum Temperature', 'Max Temp',(10,40))

In [ ]:
#Perform Linear Regression on the Maximum Temperature for the Southern Hemisphere
# Linear regression on the South Hemisphere
x_values = southern_hemi_df["Lat"]
y_values = southern_hemi_df["Max Temp"]
# Call the function.
plot_linear_regression(x_values, y_values,
                       'Linear Regression on the Northern Hemisphere \
                        for Maximum Temperature', 'Max Temp',(-50,90))

The correlation between the latitude and the maximum temperature is strong to very strong because the r-value is less than –0.7 for the Northern Hemisphere and greater than 0.7 for the Southern Hemisphere, as shown by the plots here. This means that as we approach the equator, 0° latitude, the temperatures become warmer. And when we are further from the equator the temperatures become cooler. Check the r-values for your plots. 

In [ ]:
#4.3 Find the Correlation Between Latitude and Percent Humidity

In [ ]:
# Linear regression on the Northern Hemisphere
x_values = northern_hemi_df["Lat"]
y_values = northern_hemi_df["Humidity"]
# Call the function.
plot_linear_regression(x_values, y_values,
                       'Linear Regression on the Northern Hemisphere \
                        for % Humidity', '% Humidity',(40,10))

In [ ]:
#Perform Linear Regression on the Percent Humidity for the Southern Hemisphere
# Linear regression on the Southern Hemisphere
x_values = southern_hemi_df["Lat"]
y_values = southern_hemi_df["Humidity"]
# Call the function.
plot_linear_regression(x_values, y_values,
                       'Linear Regression on the Southern Hemisphere \
                        for % Humidity', '% Humidity',(-50,15))

The correlation between the latitude and percent humidity is very low because the r-value is less than 0.04 for the Northern and Southern Hemispheres for the plots shown here. This means that percent humidity is unpredictable due to changing weather patterns that can increase or decrease percent humidity. Check the r-values for your plots. 

In [ ]:
#4.4 Find the Correlation Between Latitude and Percent Cloudiness

In [ ]:
# Linear regression on the Northern Hemisphere
x_values = northern_hemi_df["Lat"]
y_values = northern_hemi_df["Cloudiness"]
# Call the function.
plot_linear_regression(x_values, y_values,
                       'Linear Regression on the Northern Hemisphere \
                        for % Cloudiness', '% Cloudiness',(10,40))

In [ ]:
# Linear regression on the Southern Hemisphere
x_values = southern_hemi_df["Lat"]
y_values = southern_hemi_df["Cloudiness"]
# Call the function.
plot_linear_regression(x_values, y_values,
                       'Linear Regression on the Southern Hemisphere \
                        for % Cloudiness', '% Cloudiness',(-50,60))

The correlation between the latitude and percent cloudiness is very low because the r-value is less than –0.09 for the Northern Hemisphere and less than –0.02 for the Southern Hemisphere for the plots shown here. This means that cloudiness is unpredictable due to changing weather patterns that can increase or decrease percent cloudiness. Check the r-values for your plots. 

In [ ]:
#4.5  Find the Correlation Between Latitude and Wind Speed

In [ ]:
# Linear regression on the Northern Hemisphere
x_values = northern_hemi_df["Lat"]
y_values = northern_hemi_df["Wind Speed"]
# Call the function.
plot_linear_regression(x_values, y_values,
                       'Linear Regression on the Northern Hemisphere \
                        for Wind Speed', 'Wind Speed',(40,35))

In [ ]:
# Linear regression on the Southern Hemisphere
x_values = southern_hemi_df["Lat"]
y_values = southern_hemi_df["Wind Speed"]
# Call the function.
plot_linear_regression(x_values, y_values,
                       'Linear Regression on the Southern Hemisphere \
                        for Wind Speed', 'Wind Speed',(-50,35))

The correlation between the latitude and wind speed is very low because the r-value is less  than –0.07 for the Northern Hemisphere and less than –0.3 for the Southern Hemisphere for the plots shown here. This means that wind speed is unpredictable due to changing weather patterns that can increase or decrease wind speed. Check the r-values for your plots. 

# 5 Set Up Google Maps and Places API


In [ ]:
#5.2 Create Heatmaps for Weather Parameters

In [30]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests


# Import the API key.
from config1 import g_key

In [31]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Tuktoyaktuk,CA,2021-04-23 03:15:04,69.4541,-133.0374,12.20,73,20,3.44
1,1,Hobart,AU,2021-04-23 03:13:29,-42.8794,147.3294,57.20,44,75,16.11
2,2,Rikitea,PF,2021-04-23 03:15:04,-23.1203,-134.9692,77.99,77,100,13.85
3,3,Qui Nhon,VN,2021-04-23 03:15:05,13.7667,109.2333,84.58,71,18,7.67
4,4,Qaanaaq,GL,2021-04-23 03:15:05,77.4840,-69.3632,18.18,92,100,10.13


One caveat to using gmaps: The data we use for any mapping must be either an integer or a floating-point decimal number. Let's check the data types for the columns of our DataFrame.

use dtypes to get the datatypes!

In [32]:
#confirm data types for the colums are intergers or floating point decimals
#get data types
city_data_df.dtypes

City_ID         int64
City           object
Country        object
Date           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
dtype: object

In [33]:
#Create a Maximum Temperature Heatmap
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [ ]:
city_data_df.head()

In [34]:
#to remove negative temps, use a for loop to iterate through max_temp and add temps > 0F to a new list
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))

In [35]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]

# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]

# Assign the figure variable.
fig = gmaps.figure()

# Assign the heatmap variable.|
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp)


# Add the heatmap layer.
fig.add_layer(heat_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

!pip uninstall gmaps

!jupyter nbextension list

!jupyter nbextension enable --py --sys-prefix gmaps

!jupyter nbextension enable --py --sys-prefix widgetsnbextension

In [29]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [36]:
#5.3 Vacation Criterea
#create an input statement to prompt user to choose  temp
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 80


In [37]:
#create a new data frame using loc on city_data_df to filter on max temp
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
2,2,Rikitea,PF,2021-04-23 03:15:04,-23.1203,-134.9692,77.99,77,100,13.85
18,18,Puerto Ayora,EC,2021-04-23 03:10:31,-0.7393,-90.3518,77.00,94,38,4.00
43,43,Pitimbu,BR,2021-04-23 03:15:18,-7.4706,-34.8086,75.20,94,0,2.30
48,48,Airai,TL,2021-04-23 03:15:20,-8.9266,125.4092,76.05,47,29,2.08
94,94,Kapaa,US,2021-04-23 03:15:35,22.0752,-159.3190,78.80,69,40,8.05
113,113,Saint-Philippe,RE,2021-04-23 03:15:42,-21.3585,55.7679,77.00,78,90,11.50
119,119,Carolina,PR,2021-04-23 03:15:43,18.3808,-65.9574,77.00,88,20,6.91
126,126,Olinda,BR,2021-04-23 03:15:46,-8.0089,-34.8553,78.80,73,40,2.30
128,128,Cayenne,GF,2021-04-23 03:15:47,4.9333,-52.3333,78.01,94,75,8.05
132,132,Avarua,CK,2021-04-23 03:15:48,-21.2078,-159.7750,78.80,78,75,20.71


In [39]:
#Check for null values
preferred_cities_df.count()

City_ID       36
City          36
Country       36
Date          36
Lat           36
Lng           36
Max Temp      36
Humidity      36
Cloudiness    36
Wind Speed    36
dtype: int64

In [41]:
preferred_cities_df.notnull().sum()

City_ID       36
City          36
Country       36
Date          36
Lat           36
Lng           36
Max Temp      36
Humidity      36
Cloudiness    36
Wind Speed    36
dtype: int64

Depending on the time of year and the seasons, you might have to adjust the minimum and maximum temperature to get enough cities.
It is a good idea to keep the number of cities to fewer than 200 to make it easier to plot the markers on the heatmap.
If you have some rows with null values, you'll need to drop them using the dropna() method at the end of your filtering statement when you are creating the new DataFrame.

In [42]:
#5.4 Map Vacation Criteria
#Get travel destinations
#create a new data frame for data needed to create a heatmap and pop up markers
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
2,Rikitea,PF,77.99,-23.1203,-134.9692,
18,Puerto Ayora,EC,77.00,-0.7393,-90.3518,
43,Pitimbu,BR,75.20,-7.4706,-34.8086,
48,Airai,TL,76.05,-8.9266,125.4092,
94,Kapaa,US,78.80,22.0752,-159.3190,
113,Saint-Philippe,RE,77.00,-21.3585,55.7679,
119,Carolina,PR,77.00,18.3808,-65.9574,
126,Olinda,BR,78.80,-8.0089,-34.8553,
128,Cayenne,GF,78.01,4.9333,-52.3333,
132,Avarua,CK,78.80,-21.2078,-159.7750,


In [43]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [49]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
# Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.


In [50]:
hotel_df

,City,Country,Max Temp,Lat,Lng,Hotel Name
2,Rikitea,PF,77.99,-23.1203,-134.9692,Pension Maro'i
18,Puerto Ayora,EC,77.00,-0.7393,-90.3518,Finch Bay Galapagos Hotel
43,Pitimbu,BR,75.20,-7.4706,-34.8086,Reserva do Abiaí
48,Airai,TL,76.05,-8.9266,125.4092,Juvinal Martin
94,Kapaa,US,78.80,22.0752,-159.3190,Sheraton Kauai Resort at Coconut Beach
113,Saint-Philippe,RE,77.00,-21.3585,55.7679,"Chambres d'hôte ""La Trinité"""
119,Carolina,PR,77.00,18.3808,-65.9574,Motel Fantasy Star
126,Olinda,BR,78.80,-8.0089,-34.8553,Hotel Albergue de Olinda
128,Cayenne,GF,78.01,4.9333,-52.3333,Hôtel Le Dronmi
132,Avarua,CK,78.80,-21.2078,-159.7750,Paradise Inn


In [51]:
#Create a Max Temp Heatmap from a Hotel Dataframe
# Add a heatmap of temperature for the vacation spots.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [52]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [53]:
info_box_template = """
<dl>
<dt>Name</dt><dd>{column1}</dd>
<dt>Another name</dt><dd>{column2}</dd>
</dl>
"""

In [54]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

In [55]:
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [56]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))